In [ ]:
import pandas as pd
import numpy as np
import torch
from torch.utils.data import DataLoader, TensorDataset, random_split
import torch.nn as nn
import torch.optim as optim

In [ ]:
data = pd.read_csv('/content/manuel_surus_data.csv')

In [ ]:
data = data.sample(frac=1).reset_index(drop=True)

In [ ]:
X = data[['yaw_acisi', 'dx', 'dy', 'atan2_dx_dy', 'lidar_on', 'trafik_isik']].values
y = data[['hiz', 'direksiyon_aci']].values

In [ ]:

X_tensor = torch.tensor(X, dtype=torch.float32)
y_tensor = torch.tensor(y, dtype=torch.float32)

In [ ]:

dataset = TensorDataset(X_tensor, y_tensor)

In [ ]:

train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=10, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=10, shuffle=False)

In [ ]:

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.layer1 = nn.Linear(6, 32)
        self.layer2 = nn.Linear(32, 16)
        self.output = nn.Linear(16, 2)

    def forward(self, x):
        x = torch.relu(self.layer1(x))
        x = torch.relu(self.layer2(x))
        x = self.output(x)
        return x

model = NeuralNetwork()

In [ ]:

criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:

num_epochs = 25
for epoch in range(num_epochs):
    model.train()
    for batch_X, batch_y in train_loader:
        optimizer.zero_grad()
        outputs = model(batch_X)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

In [ ]:
model.eval()
test_loss = 0
with torch.no_grad():
    for batch_X, batch_y in test_loader:
        outputs = model(batch_X)
        loss = criterion(outputs, batch_y)
        test_loss += loss.item()
test_loss /= len(test_loader)
print(f'Model test loss: {test_loss:.4f}')

In [ ]:
torch.save(model, 'model.pth')

In [ ]:
model_im = torch.load('model.pth')

# Değerlendirme moduna al
model_im.eval()

In [ ]:
# 'yaw_acisi', 'dx', 'dy', 'atan2_dx_dy', 'lidar_on', 'trafik_isik'
input_dizi = [1, 1, 1, 1, 1, 1]

input_tensor = torch.tensor(input_dizi, dtype=torch.float32).unsqueeze(0)

with torch.no_grad():
    output = model_im(input_tensor)

print(f'Girdi: {input_dizi}')
print(f'Çıktı: {output}')